In [1]:
import pandas as pd
import numpy as np

In [3]:
df= pd.DataFrame({'key1': ['a','a','b','b','a'], 
                 'key2': ['one','two','one','two','one'],
                 'data1': np.random.randn(5),
                 'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.215149,-1.461023
1,a,two,0.140060,0.416803
2,b,one,0.551210,0.425669
3,b,two,-0.949704,0.537058
4,a,one,0.073151,0.105777


In [5]:
# series로 밷고 싶을 배
df.groupby('key1')['data1'].mean()

key1
a    0.142787
b   -0.199247
Name: data1, dtype: float64

In [6]:
# DF로 추출
df.groupby('key1')[['data1']].mean()

,data1
key1,
a,0.142787
b,-0.199247


In [8]:
df['data1']

0    0.215149
1    0.140060
2    0.551210
3   -0.949704
4    0.073151
Name: data1, dtype: float64

In [10]:
df[['data1']]

,data1
0,0.215149
1,0.140060
2,0.551210
3,-0.949704
4,0.073151


# 1. Split_Apply_Combine(Aggregation)

> Aggregation
- groupby로 흩어진 dataframe을 
- scalar/array/dataframe로 변환하여 취합하는 과정


In [11]:
np.random.seed(0)


In [12]:
np.random.uniform(50,100,10).astype(np.int)

array([77, 85, 80, 77, 71, 82, 71, 94, 98, 69])

In [13]:
df1 = pd.DataFrame(_.reshape(5, 2),columns= ['exam1','exam2'])
df1

,exam1,exam2
0,77,85
1,80,77
2,71,82
3,71,94
4,98,69


In [14]:
df2 = pd.DataFrame(['a','b','a','b','ab'], columns = ['blood_type'])
df2

,blood_type
0,a
1,b
2,a
3,b
4,ab


In [15]:

df3 = pd.concat((df1, df2), axis = 1)
df3

,exam1,exam2,blood_type
0,77,85,a
1,80,77,b
2,71,82,a
3,71,94,b
4,98,69,ab


In [16]:
df3.groupby('blood_type').apply(sum)['exam1']

blood_type
a     148
ab     98
b     151
Name: exam1, dtype: int64

In [21]:
df3.groupby('blood_type').apply(np.mean)

,exam1,exam2
blood_type,,
a,74.0,83.5
ab,98.0,69.0
b,75.5,85.5


In [23]:
def custom_mean(data_frame):
    return data_frame.mean(axis = 0)

df3.groupby('blood_type').apply(custom_mean)

,exam1,exam2
blood_type,,
a,74.0,83.5
ab,98.0,69.0
b,75.5,85.5


In [24]:
df3.describe().round(1)

,exam1,exam2
count,5.0,5.0
mean,79.4,81.4
std,11.1,9.3
min,71.0,69.0
25%,71.0,77.0
50%,77.0,82.0
75%,80.0,85.0
max,98.0,94.0


In [28]:
df5 = pd.DataFrame(['m','f','m','m','f'], columns = ['gender'])
df4 = pd.concat((df3, df5), axis = 1)
df4

,exam1,exam2,blood_type,gender
0,77,85,a,m
1,80,77,b,f
2,71,82,a,m
3,71,94,b,m
4,98,69,ab,f


In [31]:
df4.groupby(['blood_type','gender']).apply(custom_mean)

exam1  exam2
blood_type gender              
a          m        74.0   83.5
ab         f        98.0   69.0
b          f        80.0   77.0
           m        71.0   94.0

In [32]:
_.index

MultiIndex([( 'a', 'm'),
            ('ab', 'f'),
            ( 'b', 'f'),
            ( 'b', 'm')],
           names=['blood_type', 'gender'])

In [34]:
df4.groupby(['blood_type','gender']).apply(np.mean)

exam1  exam2
blood_type gender              
a          m        74.0   83.5
ab         f        98.0   69.0
b          f        80.0   77.0
           m        71.0   94.0

In [36]:
# multi_index된 최종 결과물을 tidy한 포멧으로 바꾸고 싶다면
df4.groupby(['blood_type','gender']).apply(np.mean).reset_index()

,blood_type,gender,exam1,exam2
0,a,m,74.0,83.5
1,ab,f,98.0,69.0
2,b,f,80.0,77.0
3,b,m,71.0,94.0


In [38]:
df3

,exam1,exam2,blood_type
0,77,85,a
1,80,77,b
2,71,82,a
3,71,94,b
4,98,69,ab


In [37]:
pd.cut(df3.exam1, 2)

0    (70.973, 84.5]
1    (70.973, 84.5]
2    (70.973, 84.5]
3    (70.973, 84.5]
4      (84.5, 98.0]
Name: exam1, dtype: category
Categories (2, interval[float64]): [(70.973, 84.5] < (84.5, 98.0]]

In [40]:
pd.qcut(df3.exam1, 2)

0    (70.999, 77.0]
1      (77.0, 98.0]
2    (70.999, 77.0]
3    (70.999, 77.0]
4      (77.0, 98.0]
Name: exam1, dtype: category
Categories (2, interval[float64]): [(70.999, 77.0] < (77.0, 98.0]]

In [41]:
pd.qcut(df3.exam1, 2, labels = ['low', 'high'])

0     low
1    high
2     low
3     low
4    high
Name: exam1, dtype: category
Categories (2, object): [low < high]

In [42]:
# 시험 1의 상위권 성적의 학생들이 시험2에서도 유지되는자?
df3.exam2.groupby(_).mean()

exam1
low     87
high    73
Name: exam2, dtype: int32

In [47]:
df4

,exam1,exam2,blood_type,gender
0,NaN,85,a,m
1,80.0,77,b,f
2,NaN,82,a,m
3,71.0,94,b,m
4,NaN,69,ab,f


In [48]:
# groupby 연산으로 결측치 채우기
df4.exam1.iloc[::2]

0   NaN
2   NaN
4   NaN
Name: exam1, dtype: float64

In [50]:
df4.exam1.iloc[::2] = np.nan
df4

,exam1,exam2,blood_type,gender
0,NaN,85,a,m
1,80.0,77,b,f
2,NaN,82,a,m
3,71.0,94,b,m
4,NaN,69,ab,f


In [51]:
# 앞의 값으로 null값 처리
df4.groupby('gender').apply(lambda x: x.fillna(x.mean()))

exam1  exam2 blood_type gender
gender                                  
f      1   80.0     77          b      f
       4   80.0     69         ab      f
m      0   71.0     85          a      m
       2   71.0     82          a      m
       3   71.0     94          b      m

In [52]:
temp= _
temp.index = temp.index.droplevel()
temp

,exam1,exam2,blood_type,gender
1,80.0,77,b,f
4,80.0,69,ab,f
0,71.0,85,a,m
2,71.0,82,a,m
3,71.0,94,b,m


In [54]:
temp.sort_index()

,exam1,exam2,blood_type,gender
0,71.0,85,a,m
1,80.0,77,b,f
2,71.0,82,a,m
3,71.0,94,b,m
4,80.0,69,ab,f


# 2. Pivot Tables

> Pivot Tables
- 데이터 분석을 할 때 리포트 형식의 data summarization도구
- groupby와 multiIndex를 잘 활용하여 작성
- pivot_table()함수 이용

In [56]:
# data set불러오기

url="https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/tips.csv" 
tips = pd.read_csv(url) 
tips.head()


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [57]:
tips.pivot_table(index = ['sex','smoker'])

size       tip  total_bill
sex    smoker                                
Female No      2.592593  2.773519   18.105185
       Yes     2.242424  2.931515   17.977879
Male   No      2.711340  3.113402   19.791237
       Yes     2.500000  3.051167   22.284500

In [62]:

tips.pivot_table(['total_bill', 'size'],\
index= ['sex','day'], columns = ['smoker']).round(1)

size      total_bill      
smoker        No  Yes         No   Yes
sex    day                            
Female Fri   2.5  2.0       19.4  12.7
       Sat   2.3  2.2       19.0  20.3
       Sun   3.1  2.5       20.8  16.5
       Thur  2.5  2.4       16.0  19.2
Male   Fri   2.0  2.1       17.5  20.5
       Sat   2.7  2.6       19.9  21.8
       Sun   2.9  2.6       20.4  26.1
       Thur  2.5  2.3       18.5  19.2

In [64]:
tips.pivot_table(['total_bill','size'], index = ['sex','day'], columns = 'smoker', margins = True).round(1)

size           total_bill            
smoker        No  Yes  All         No   Yes   All
sex    day                                       
Female Fri   2.5  2.0  2.1       19.4  12.7  14.1
       Sat   2.3  2.2  2.2       19.0  20.3  19.7
       Sun   3.1  2.5  2.9       20.8  16.5  19.9
       Thur  2.5  2.4  2.5       16.0  19.2  16.7
Male   Fri   2.0  2.1  2.1       17.5  20.5  19.9
       Sat   2.7  2.6  2.6       19.9  21.8  20.8
       Sun   2.9  2.6  2.8       20.4  26.1  21.9
       Thur  2.5  2.3  2.4       18.5  19.2  18.7
All          2.7  2.4  2.6       19.2  20.8  19.8

In [67]:
tips.pivot_table(['total_bill', 'size'], index = ['sex','day'], columns = 'smoker',
                margins = True, aggfunc= 'count').round(1)

size          total_bill         
smoker        No Yes  All         No Yes  All
sex    day                                   
Female Fri     2   7    9          2   7    9
       Sat    13  15   28         13  15   28
       Sun    14   4   18         14   4   18
       Thur   25   7   32         25   7   32
Male   Fri     2   8   10          2   8   10
       Sat    32  27   59         32  27   59
       Sun    43  15   58         43  15   58
       Thur   20  10   30         20  10   30
All          151  93  244        151  93  244